In [36]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf
from datetime import datetime
from datetime import timedelta

In [37]:
def fetch_stocks():
    filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    # filters = ['sh_avgvol_o2000', 'sh_price_o20']
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [38]:
def cal_VRRS(ticker, ref, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * 100

In [39]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = 100 * ((close - highh) / (highh - lowl))
    return wr

In [40]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        if len(x) > 1:
            return float(x.replace('B', '')) * 1000000000
        return 1000000000.0
    if 'T' in x:
        if len(x) > 1:
            return float(x.replace('T', '')) * 1000000000000
        return 1000000000000000.0

    return 0.0

In [41]:
ticker_dict = {}

In [42]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.58it/s]


In [47]:
start_date = "2017-01-01"
# end_date = "2022-05-28"

end_date = datetime.today().date()

spy = yf.Ticker("SPY")
spy = spy.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()

for stock in stock_list:
    
    ticker = stock['Ticker']
    
    avg_volume = value_to_float(stock['Avg Volume'])
    
    if avg_volume > 5000000:

        stock = yf.Ticker(ticker)    

        stock_close = stock.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()
        stock_now = stock.history(start=start_date, end=end_date, interval='5d').dropna()
        
        # williams = get_wr(stock_now['High'], stock_now['Low'], stock_now['Close'], 2).iloc[-1]

        # Calculate the VRRS indicator:
        rolling_length = 21 

        vrrs = cal_VRRS(stock_close, spy, rolling_length).iloc[-1]
        week_ago_VRRS = cal_VRRS(stock_close.iloc[:-1], spy.iloc[:-1], rolling_length).iloc[-1]
        
        difference = float(vrrs) - float(week_ago_VRRS)

        if abs(difference) > 4:
            print(ticker, round(vrrs, 2), round(difference, 2))

LABU -11.09 23.65
UVXY -9.58 -9.92
PSQ 12.75 -7.28
SH 12.58 -5.07
PDBC 5.31 -8.92
SLV -9.54 -7.93
SPXU 18.96 -10.86
UNG 3.21 -10.14
QID 15.59 -12.42
SPXS 18.83 -10.95
GDX -12.77 -5.98
GDXJ -17.75 -6.46
KWEB 12.45 -4.62
IAU -0.18 -6.27
SDOW 16.78 -6.87
TNA -21.05 4.62
TZA 17.47 -13.56
XME -11.12 -5.88
ERX -7.66 -8.6
QLD -13.4 4.73
SDS 16.51 -8.05
SQQQ 15.35 -17.51
SOXS 29.87 -11.1
XLE 1.77 -5.68
USO 10.03 -9.43
XBI 10.22 9.71
XOP -0.93 -7.26
GLD -0.27 -6.27
